# Seminar 3

## RSA (Rivest, Shamir and Adleman)

This example demonstrates RSA public-key cryptography in an easy-to-follow manner. It works on integers alone, and uses much smaller numbers for the sake of clarity. 

### First we pick our primes. These will determine our keys.

Pick P,Q,and E such that:

    1. P and Q are prime; picked at random.
    2. 1 < E < (P-1)*(Q-1) and E is co-prime with (P-1)*(Q-1)

In [15]:
def gcd(a,b):
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a

def primitive_root(modulo):
    required_set = set(num for num in range (1, modulo) if gcd(num, modulo) == 1)
    for g in range(1, modulo):
        actual_set = set(pow(g, powers) % modulo for powers in range (1, modulo))
        if required_set == actual_set:
            print(g)

In [16]:
primitive_root(11)

2
6
7
8


In [6]:
def primfacs(n): #Факторизация целых чисел
    i = 2
    primfac = []
    while i * i <= n:
        while n % i == 0:
            primfac.append(i)
            n = n / i
        i = i + 1
    if n > 1:
        primfac.append(n)
    return primfac

In [ ]:
N = 55
E = 33
Q,P = primfacs(N)

In [20]:
P=11    # First prime
Q=5    # Second prime
E=33    # usually a constant; 0x10001 is common, prime is best

### Next, some functions we'll need in a moment:

In [21]:
def isPrime(x):
    if x%2==0 and x>2: return False     # False for all even numbers
    i=3                                 # we don't divide by 1 or 2
    sqrt=x**.5                          
    while i<sqrt:
        if x%i==0: return False
        i+=2
    return True

# Part of find_inverse below
# See: http://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
def eea(a,b):
    if b==0:return (1,0)
    (q,r) = (a//b,a%b)
    (s,t) = eea(b,r)
    return (t, s-(q*t) )

# Find the multiplicative inverse of x (mod y)
# see: http://en.wikipedia.org/wiki/Modular_multiplicative_inverse
def find_inverse(x,y):
    inv = eea(x,y)[0]
    if inv < 1: inv += y #we only want positive values
    return inv

In [22]:
eea(6,5)

(1, -1)

### Make sure the numbers we picked above are valid.

In [23]:
if not isPrime(P): raise Exception("P (%i) is not prime" % (P,))
if not isPrime(Q): raise Exception("Q (%i) is not prime" % (Q,))

T=(P-1)*(Q-1) # Euler's totient (intermediate result)
# Assuming E is prime, we just have to check against T
if E<1 or E > T: raise Exception("E must be > 1 and < T")
if T%E==0: raise Exception("E is not coprime with T")

### Derive our keys.

In [24]:
# Product of P and Q is our modulus; the part determines as the "key size".
MOD=P*Q

# Private exponent is inverse of public exponent with respect to (mod T)
D = find_inverse(E,T)
print(D)

17


The modulus is always needed, while either E or D is the exponent, depending on which key we're using. D is much harder for an adversary to derive, so we call that one the "private" key.
Note. See https://www.python-course.eu/python3_formatted_output.php for print format details.

In [25]:
print("public key: (MOD: %i, E: %i)" % (MOD,E))
print("private key: (MOD: %i, D: %i)" % (MOD,D))

public key: (MOD: 55, E: 33)
private key: (MOD: 55, D: 17)


Note that P, Q, and T can now be discarded, but they're usually kept around so that a more efficient encryption algorithm can be used. http://en.wikipedia.org/wiki/RSA#Using_the_Chinese_remainder_algorithm

### We have our keys, let's do some encryption

Here we only focus on whether you're applying the private key or applying the public key, since either one will reverse the other. 

In [30]:
message = int(41)
print("Initial message: %i" % message)

# Note that the pow() built-in does modulo exponentation. That's handy, since it saves us having to
# implement that ablity.
# http://en.wikipedia.org/wiki/Modular_exponentiation

if (message >= MOD):
    raise Exception("Message must be < MOD")

key = E
encrypted_message = pow(message,key,MOD)
print("Encrypted_message: %i" % encrypted_message)

key = D
decrypted_message = pow(encrypted_message,key,MOD) #encrypt/decrypt using this ONE command. Surprisingly simple.
print("Decrypted_message: %i" % decrypted_message)

Initial message: 41
Encrypted_message: 6
Decrypted_message: 41


In [ ]:
key = E
encrypted_message = pow(message,key,MOD)
print("Encrypted_message: %i" % encrypted_message)

key = D
decrypted_message = pow(encrypted_message,key,MOD) #encrypt/decrypt using this ONE command. Surprisingly simple.
print("Decrypted_message: %i" % decrypted_message)

In [33]:
encripted=(41**33)%55

In [34]:
encripted

6

Now let's find digital signatures for the same message, but please do it by yourself

In [28]:
message = int(13)
print("digital_signature: %i" % message)

# Note that the pow() built-in does modulo exponentation. That's handy, since it saves us having to
# implement that ablity.
# http://en.wikipedia.org/wiki/Modular_exponentiation

if (message >= MOD):
    raise Exception("Message must be < MOD")

key = D
encrypted_message = pow(message,key,MOD)
print("Encrypted_message: %i" % encrypted_message)

key = E
decrypted_message = pow(encrypted_message,key,MOD) #encrypt/decrypt using this ONE command. Surprisingly simple.
print("digital_signature: %i" % decrypted_message)

digital_signature: 13
Encrypted_message: 18
digital_signature: 13


What are the differences between digital signatures and encryption in RSA? 

## El-Gamal protocol

Decrypt the El-Gamal message if open key is equal to p=13, g=6, y=8

Encryped message is equal to (11,11)

### See the black board

Find digital signature using El-Gamal protocol with parameters 



Open key p=11, g=7, y=8



Message M=6



Random parameter k=3

### See the black board

## Problem of generating big prime number

Fermat's little theorem states that if p is a prime number, then for any integer a, the number ap − a is an integer multiple of p. In the notation of modular arithmetic, this is expressed as

$$a^p \equiv a \pmod p$$

If a is not divisible by p, Fermat's little theorem is equivalent to the statement that $a^{p − 1} − 1$ is an integer multiple of p, or in symbols:

$$a^{p-1} \equiv 1 \pmod p$$





In [10]:
# Python implementation of Fermat's primality test to generate prime numbers of any bit length.

from random import randint

def is_prime(num, test_count):
    if num == 1:
        return False
    if test_count >= num:
        test_count = num - 1
    for x in range(test_count):
        val = randint(1, num - 1)
        if pow(val, num-1, num) != 1:
            return False
    return True

def generate_big_prime(n, test_count=1000):
    found_prime = False
    while not found_prime:
        p = randint(2**(n-1), 2**n)
        print("p ", p,'\n')
        if is_prime(p, test_count):
            return p

In [11]:
n = 20
random_prime = generate_big_prime(n)
print(random_prime)
print("{0:b}".format(random_prime))
# Generates a random prime number of length n bits

p  677863 

p  912512 

p  875183 

875183
11010101101010101111


In [1]:
def __gcd(a,b): 
  
    if(b == 0): 
        return a 
    else: 
        return __gcd(b, a % b) 
      
# To compute x^y under modulo m 
def power(x,y,m): 
  
    if (y == 0): 
        return 1
    p = power(x, y // 2, m) % m 
    p = (p * p) % m 
   
    return p if(y % 2 == 0) else  (x * p) % m 
  
# Function to find modular 
# inverse of a under modulo m 
# Assumption: m is prime 
def modInverse(a,m): 
  
    if (__gcd(a, m) != 1): 
        print("Inverse doesn't exist") 
   
    else: 
   
        # If a and m are relatively prime, then 
        # modulo inverse is a^(m-2) mode m 
        print("Modular multiplicative inverse is ", 
             power(a, m - 2, m)) 


In [5]:
modInverse(1,11)

Modular multiplicative inverse is  1


Note 1: All variables are 32 bit unsigned integers and addition is done modulo 2^32

Note 2: For each round, there is one round constant k[i] and one entry in the message schedule array w[i], 0 ≤ i ≤ 63

Note 3: The compression function uses 8 working variables, a through h

Note 4: Big-endian convention is used when expressing the constants in this pseudocode,
    and when parsing message block data from bytes to words, for example,
    the first word of the input message "abc" after padding is 0x61626380

Initialize hash values:
(first 32 bits of the fractional parts of the square roots of the first 8 primes 2..19):

h0 := 0x6a09e667
h1 := 0xbb67ae85
h2 := 0x3c6ef372
h3 := 0xa54ff53a
h4 := 0x510e527f
h5 := 0x9b05688c
h6 := 0x1f83d9ab
h7 := 0x5be0cd19

Initialize array of round constants:
(first 32 bits of the fractional parts of the cube roots of the first 64 primes 2..311):

k[0..63] :=
   0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
   0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
   0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
   0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
   0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
   0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
   0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
   0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2

Pre-processing (Padding):
begin with the original message of length L bits
append a single '1' bit
append K '0' bits, where K is the minimum number >= 0 such that L + 1 + K + 64 is a multiple of 512
append L as a 64-bit big-endian integer, making the total post-processed length a multiple of 512 bits

Process the message in successive 512-bit chunks:
break message into 512-bit chunks
for each chunk
    create a 64-entry message schedule array w[0..63] of 32-bit words
    (The initial values in w[0..63] don't matter, so many implementations zero them here)
    copy chunk into first 16 words w[0..15] of the message schedule array

    Extend the first 16 words into the remaining 48 words w[16..63] of the message schedule array:
    for i from 16 to 63
        s0 := (w[i-15] rightrotate 7) xor (w[i-15] rightrotate 18) xor (w[i-15] rightshift 3)
        s1 := (w[i-2] rightrotate 17) xor (w[i-2] rightrotate 19) xor (w[i-2] rightshift 10)
        w[i] := w[i-16] + s0 + w[i-7] + s1

    Initialize working variables to current hash value:
    a := h0
    b := h1
    c := h2
    d := h3
    e := h4
    f := h5
    g := h6
    h := h7

    Compression function main loop:
    for i from 0 to 63
        S1 := (e rightrotate 6) xor (e rightrotate 11) xor (e rightrotate 25)
        ch := (e and f) xor ((not e) and g)
        temp1 := h + S1 + ch + k[i] + w[i]
        S0 := (a rightrotate 2) xor (a rightrotate 13) xor (a rightrotate 22)
        maj := (a and b) xor (a and c) xor (b and c)
        temp2 := S0 + maj
 
        h := g
        g := f
        f := e
        e := d + temp1
        d := c
        c := b
        b := a
        a := temp1 + temp2

    Add the compressed chunk to the current hash value:
    h0 := h0 + a
    h1 := h1 + b
    h2 := h2 + c
    h3 := h3 + d
    h4 := h4 + e
    h5 := h5 + f
    h6 := h6 + g
    h7 := h7 + h

Produce the final hash value (big-endian):
digest := hash := h0 append h1 append h2 append h3 append h4 append h5 append h6 append h7